<a href="https://colab.research.google.com/github/grabuffo/BrainStim_ANN_fMRI_HCP/blob/main/notebooks/HCP_ANN_Connectiviy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================
# 🧠 Compute EC(t) for each subject using NPI.model_ECt
# ============================================

# --- 1️⃣ Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- 2️⃣ Clone GitHub repo (contains src/NPI.py) ---
!rm -rf /content/BrainStim_ANN_fMRI_HCP
!git clone https://github.com/grabuffo/BrainStim_ANN_fMRI_HCP.git

# --- 3️⃣ Define paths ---
import os, sys, gc
repo_dir    = "/content/BrainStim_ANN_fMRI_HCP"
data_dir    = "/content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data"
preproc_dir = os.path.join(data_dir, "preprocessed_subjects")
models_dir  = os.path.join(preproc_dir, "trained_models_MLP")
ects_dir    = os.path.join(preproc_dir, "ECts_MLP")
os.makedirs(ects_dir, exist_ok=True)

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# --- 4️⃣ Imports ---
import numpy as np
import torch
import torch.serialization
from src import NPI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Repo loaded from:", repo_dir)
print("Using device:", device)

# --- 5️⃣ Choose which subjects to process ---
# either specify manually:
subjects = ["id_100206"]
# or automatically detect all
# subjects = sorted({fn.split("_signals.npy")[0]
#                    for fn in os.listdir(preproc_dir)
#                    if fn.endswith("_signals.npy")})

# --- 6️⃣ Allowlist your model classes (needed for PyTorch ≥2.6) ---
torch.serialization.add_safe_globals(
    [NPI.ANN_MLP, NPI.ANN_CNN, NPI.ANN_RNN, NPI.ANN_VAR]
)

# --- 7️⃣ Define helper to load model (full model or checkpoint) ---
def load_model(model_path, inputs, targets):
    ckpt = torch.load(model_path, map_location=device, weights_only=False)
    if hasattr(ckpt, "eval"):  # full model saved with torch.save(model)
        model = ckpt.to(device)
        model.eval()
        return model
    if isinstance(ckpt, dict) and "model_state_dict" in ckpt:
        method = ckpt.get("method", "MLP")
        ROI_num = ckpt.get("ROI_num", targets.shape[-1])
        using_steps = ckpt.get("using_steps", inputs.shape[-2] if inputs.ndim > 1 else 1)
        model = NPI.build_model(method, ROI_num, using_steps).to(device)
        model.load_state_dict(ckpt["model_state_dict"])
        model.eval()
        return model
    raise ValueError("Unrecognized model file format")

# --- 8️⃣ Main EC(t) extraction loop ---
pert_strength = 0.1
ECts = {}

for sid in subjects:
    print(f"\n================ {sid} ================")

    sig_path = os.path.join(preproc_dir, f"{sid}_signals.npy")
    inp_path = os.path.join(preproc_dir, f"{sid}_inputs.npy")
    tgt_path = os.path.join(preproc_dir, f"{sid}_targets.npy")
    mdl_path = os.path.join(models_dir,  f"{sid}_MLP.pt")

    if not os.path.exists(sig_path) or not os.path.exists(mdl_path):
        print(f"❌ Missing data or model for {sid}")
        continue

    # Load fMRI windows
    Z = np.load(sig_path)             # (T, N)
    X = np.load(inp_path)             # (M, S*N)
    Y = np.load(tgt_path)             # (M, N)

    # Load model
    model = load_model(mdl_path, X, Y)
    print("🧩 Model loaded.")

    # Compute EC(t)
    EC_t = NPI.model_ECt(model, input_X=X, target_Y=Y, pert_strength=pert_strength)
    ECts[sid] = EC_t
    print(f"✅ EC(t) computed: {EC_t.shape}")

    # Save
    out_path = os.path.join(ects_dir, f"{sid}_ECt.npy")
    np.save(out_path, EC_t)
    print(f"💾 Saved EC(t) → {out_path}")

    del Z, X, Y, model, EC_t
    gc.collect(); torch.cuda.empty_cache()

print("\n🎯 All subjects processed successfully.")



Mounted at /content/drive
Cloning into 'BrainStim_ANN_fMRI_HCP'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 126 (delta 35), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (126/126), 22.47 MiB | 1.33 MiB/s, done.
Resolving deltas: 100% (35/35), done.
✅ Repo loaded from: /content/BrainStim_ANN_fMRI_HCP
Using device: cpu

================ id_100206 ================
🧩 Model loaded.
✅ EC(t) computed: (4677, 450, 450)
💾 Saved EC(t) → /content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data/preprocessed_subjects/ECts_MLP/id_100206_ECt.npy

🎯 All subjects processed successfully.


In [6]:
!ls /content/BrainStim_ANN_fMRI_HCP


notebooks  README.md  src
